In [2]:
from pythonMD import manager
from unum.units import *
import matplotlib.pyplot as plt

# Test Lennard-Jones
## Increase timestep and check energy convergence

In [5]:
myManager = manager(forces=["LJ"],boundaries=[11.3,11.3,11.3],dimensions=3)

# for dt in range(1,20,2):
#     myManager.dt = 0.01*dt
myManager.initialize(positions="coord_ex.xyz",masses=40*U,temperature=298*K)
data = myManager.run(1000,LJ={"Ar-Ar":{"sigma":3.4*ANGSTROM,"epsilon": 1.656778224E-21*J}},saveStats=1,savePositions=1,printStats=100)
totalRelativeEnergy = [abs((energy-data["totalEnergy"][0])*100/energy) for energy in data["totalEnergy"]]
totalRelativeEnergy.sort()
# print(f"maximum change in energy for dt={dt}: {totalRelativeEnergy[-1]}%")

Initializing Molecular Dynamics Simulation
Reading positions
Reading 32 atoms
Calculating Momentums using  MaxwellBoltzmann
Start running for 1000 iterations
Timestep Temperature KineticEnergy PotentialEnergy TotalEnergy
100 240.10829091018545 2992.085456348185 -10580.15807377056 -7588.0726174223755
200 162.96242528170677 2030.7399664054608 -8657.43179399096 -6626.6918275855
300 181.67766769773652 2263.958088248574 -9123.806253742547 -6859.8481654939715
400 160.42278034648913 1999.0924350097398 -8594.104743092108 -6595.01230808237
500 175.52684879393107 2187.3102735606926 -8970.591764452005 -6783.281490891312
600 190.3707080402146 2372.28554117233 -9340.592225516348 -6968.306684344018
700 165.44018673657826 2061.6163429991966 -8719.208932835587 -6657.59258983639
800 165.97540012502506 2068.285851117987 -8732.513162527654 -6664.227311409668
900 150.9563505560688 1881.1274668193957 -8358.167224564451 -6477.039757745055


In [6]:
xyz=myManager.makePositionsFile(data["positions"],"testArgon.xyz")